# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [1]:
%pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Analyzer Templates

Below is a collection of analyzer templates designed to extract fields from various input file types.

These templates are highly customizable, allowing you to modify them to suit your specific needs. For additional verified templates from Microsoft, please visit [here](../analyzer_templates/README.md).

In [2]:
extraction_templates = {
    "invoice":            ('../analyzer_templates/invoice.json',         '../data/invoice.pdf'            ),
    "chart":              ('../analyzer_templates/image_chart.json',     '../data/pieChart.jpg'           ),
    "call_recording":     ('../analyzer_templates/call_recording_analytics.json', '../data/callCenterRecording.mp3'),
    "conversation_audio": ('../analyzer_templates/conversational_audio_analytics.json', '../data/callCenterRecording.mp3'),
    "marketing_video":    ('../analyzer_templates/marketing_video.json', '../data/video.mp4'              ),
    "PostCallAnalyticsOnText":  ('../CU-Demo-Assets/analyzers/PostCallAnalyticsOnText.json', '../CU-Demo-Assets/en-us-conversation-transcripts/sample0 - Insurance - Customer calls about purchasing life insurance.wav.results.txt'),
    "PostCallAnalyticsCustomized": ('../CU-Demo-Assets/analyzers/PostCallAnalyticsCustomized.json', '../CU-Demo-Assets/en-US-mono-audio/sample4 - Healthcare - Customer calls about new policy quote.wav'),
    "PostCallAnalyticsCustomizedDE": ('../CU-Demo-Assets/analyzers/PostCallAnalyticsWithGermanResult.json', '../CU-Demo-Assets/de-DE-mono-audio/sample4 - Healthcare - Customer calls about new policy quote- German.wav'),
    "ImageTextAnalysis": ('../CU-Demo-Assets/analyzers/imageTextAnalysis.json', '../CU-Demo-Assets/en-us-images-w-text/Designer.png'),
    "MultiLanguageDemo": ('../CU-Demo-Assets/analyzers/MultiLanguageDemo.json', '../CU-Demo-Assets/pt-BR-mono-audio/sample4 - Healthcare - Customer calls about new policy quote - Brazilian.wav'),
    "MultiLanguageDemoOnText":  ('../CU-Demo-Assets/analyzers/MultiLanguageDemoOnText.json', '../CU-Demo-Assets/pt-BR-conversation-transcripts/sample6 - Financial - Customer calls about transferring funds - Brazilian.wav.results.txt'),
    "PostCallAnalyticsCustomizedMultiLanguageMoreFields":  ('../CU-Demo-Assets/analyzers/PostCallAnalyticsCustomizedMultiLanguageMoreFields.json', '../CU-Demo-Assets/de-DE-mono-audio/sample4 - Healthcare - Customer calls about new policy quote- German.wav'),
    "ACSTesting":  ('../CU-Demo-Assets/analyzers/ACSTesting.json', '../CU-Demo-Assets/en-US-mono-audio/sample4 - Healthcare - Customer calls about new policy quote.wav'),
    "ACSTestingIRCLog": ('../CU-Demo-Assets/analyzers/ACSTestingIRCLog.json', '../CU-Demo-Assets/en-us-text-examples/linux.log.txt'),
    "CarInsuranceCallAnalysis":  ('../CU-Demo-Assets/analyzers/CarInsuranceCallAnalysis.json', '../CU-Demo-Assets/en-US-mono-audio/Car Insurance Call.wav'),
    "StoryAnalysis": ('../CU-Demo-Assets/analyzers/HTML-ShortStoryAnalyzer.json', '../CU-Demo-Assets/en-us-text-examples/extended_short_story.html'),
    "StoryAnalysisWObject": ('../CU-Demo-Assets/analyzers/HTML-ShortStoryAnalyzerWObject.json', '../CU-Demo-Assets/en-us-text-examples/extended_short_story.html')
}

Specify the analyzer template you want to use and provide a name for the analyzer to be created based on the template.

In [3]:
import uuid

ANALYZER_TEMPLATE = "call_recording"
ANALYZER_ID = "cu-demo-" + str(uuid.uuid4())

(analyzer_template_path, analyzer_sample_file_path) = extraction_templates[ANALYZER_TEMPLATE]

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [4]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2024-12-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/field_extraction", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.36)'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 400
Response headers:
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'IMDS/150.870.65.1544'
    'x-ms-request-id': 'eefb97a3-5409-414e-b6be-044ab38b69e4'
    'Date': 'Sat, 03 May 2025 00:01:09 GMT'
    'Content-Length': '88'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Re

## List all analyzers created in your resource

After the analyzer is successfully created, we can use it to analyze our input files.

In [5]:
response = client.get_all_analyzers()
print(f"Number of analyzers in your resource: {len(response['value'])}")
print(f"First 3 analyzer details: {json.dumps(response['value'], indent=2)}")

Number of analyzers in your resource: 8
First 3 analyzer details: [
  {
    "analyzerId": "prebuilt-read",
    "description": "Extract content elements such as words, barcodes, and formulas from documents.",
    "config": {
      "returnDetails": true,
      "enableOcr": true,
      "enableLayout": false,
      "enableBarcode": false,
      "enableFormula": false
    },
    "warnings": [],
    "status": "undefined",
    "scenario": "document"
  },
  {
    "analyzerId": "prebuilt-layout",
    "description": "Extract various content and layout elements such as words, paragraphs, and tables from documents.",
    "config": {
      "returnDetails": true,
      "enableOcr": true,
      "enableLayout": true,
      "enableBarcode": false,
      "enableFormula": false
    },
    "warnings": [],
    "status": "undefined",
    "scenario": "document"
  },
  {
    "analyzerId": "TempTest",
    "description": "",
    "tags": {
      "projectId": "eaf9a896-8d9c-46ae-8cfd-e3f601ae388f",
      "templat

## Create Analyzer from the Template

In [6]:
response = client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzer cu-demo-23766311-333a-4d71-9cf0-f072b517acf1 create request accepted.
INFO:python.content_understanding_client:Request 3af3c1b0-9d98-4eb6-88b6-2e53c1dbdadd in progress ...
INFO:python.content_understanding_client:Request result is ready after 2.99 seconds.


{
  "id": "3af3c1b0-9d98-4eb6-88b6-2e53c1dbdadd",
  "status": "Succeeded",
  "result": {
    "analyzerId": "cu-demo-23766311-333a-4d71-9cf0-f072b517acf1",
    "description": "Sample call recording analytics",
    "createdAt": "2025-05-03T00:01:13Z",
    "lastModifiedAt": "2025-05-03T00:01:14Z",
    "config": {
      "locales": [
        "en-US"
      ],
      "returnDetails": true,
      "disableContentFiltering": false
    },
    "fieldSchema": {
      "fields": {
        "Summary": {
          "type": "string",
          "method": "generate",
          "description": "A one-paragraph summary"
        },
        "Topics": {
          "type": "array",
          "method": "generate",
          "description": "Top 5 topics mentioned",
          "items": {
            "type": "string"
          }
        },
        "Companies": {
          "type": "array",
          "method": "generate",
          "description": "List of companies mentioned",
          "items": {
            "type": "stri

# Extract Conversation (WEBVTT) from either Batch Transcription JSON or Content Understanding JSON

## Extract Fields Using the Analyzer

After the analyzer is successfully created, we can use it to analyze our input files.

In [7]:
response = client.begin_analyze(ANALYZER_ID, file_location=analyzer_sample_file_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzing file ../data/callCenterRecording.mp3 with analyzer: cu-demo-23766311-333a-4d71-9cf0-f072b517acf1
INFO:python.content_understanding_client:Request 6a70be03-2a42-4699-b717-1643536f9cd4 in progress ...
INFO:python.content_understanding_client:Request 6a70be03-2a42-4699-b717-1643536f9cd4 in progress ...
INFO:python.content_understanding_client:Request 6a70be03-2a42-4699-b717-1643536f9cd4 in progress ...
INFO:python.content_understanding_client:Request result is ready after 8.21 seconds.


{
  "id": "6a70be03-2a42-4699-b717-1643536f9cd4",
  "status": "Succeeded",
  "result": {
    "analyzerId": "cu-demo-23766311-333a-4d71-9cf0-f072b517acf1",
    "apiVersion": "2024-12-01-preview",
    "createdAt": "2025-05-03T00:01:18Z",
    "warnings": [],
    "contents": [
      {
        "markdown": "```WEBVTT\n\n00:00.080 --> 00:00.640\n<v Agent>Good day.\n\n00:00.960 --> 00:02.240\n<v Agent>Welcome to Contoso.\n\n00:02.560 --> 00:03.760\n<v Agent>My name is John Doe.\n\n00:03.920 --> 00:05.120\n<v Agent>How can I help you today?\n\n00:05.440 --> 00:06.320\n<v Customer>Yes, good day.\n\n00:06.720 --> 00:08.160\n<v Customer>My name is Maria Smith.\n\n00:08.560 --> 00:11.280\n<v Customer>I would like to inquire about my current point balance.\n\n00:11.680 --> 00:12.560\n<v Agent>No problem.\n\n00:12.880 --> 00:13.920\n<v Agent>I am happy to help.\n\n00:14.240 --> 00:16.720\n<v Agent>I need your date of birth to confirm your identity.\n\n00:17.120 --> 00:19.600\n<v Customer>It is April 

## Clean Up
Optionally, delete the sample analyzer from your resource. In typical usage scenarios, you would analyze multiple files using the same analyzer.

In [8]:
client.delete_analyzer(ANALYZER_ID)

INFO:python.content_understanding_client:Analyzer cu-demo-23766311-333a-4d71-9cf0-f072b517acf1 deleted.


<Response [204]>